## PyTorch FashionMNIST 数据集 神经网络搭建与训练
    训练模型，实现给定图片就能识别种类
    分析：
        60000张图片去训练模型，每张图片获取一个最优的参数
        1、输入层：60000-gray图片，28*28像素， X.shape=(60000,784)
        2、隐藏层：10个神经元 参数矩阵shape=(784,64) 偏置bias shape=(64,)
        3、输出层：参数矩阵shape=(64,10) 偏置bias-shape=(10,) Y输出shape=(,10)
        4、损失函数：交叉熵损失函数

### 导包

In [88]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader 
from torchvision.datasets import FashionMNIST
from torchvision.transforms import ToTensor

### 参数初始化

In [89]:
lr = 0.01
epochs = 10
BATCH_SIZE = 128

### 数据集初始化（输入层）

In [90]:
train_data = FashionMNIST("./data", train=True, download=True, transform=ToTensor())
# print(train_data[0][0].shape)
test_data = FashionMNIST("./data", train=False, download=True, transform=ToTensor())

# 数据集分成多个批次 shuffle=True 表示打乱顺序
train_dl = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

### 构建模型（隐藏层）

In [92]:
model = nn.Sequential(
    nn.Linear(784, 64),
    nn.Sigmoid(),
    nn.Linear(64, 10)
)

### 定义损失函数和优化器（隐藏层）

In [93]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr)

### 训练模型

In [94]:
# 分多轮训练
for epoch in range(epochs):
    # 提取训练数据
    for x, y in train_dl:
        # torch.Size([1000, 1, 28, 28]) torch.Size([1000])
        # print(x.shape, y.shape)
        # ① 前向传播
        y_hat = model(x.reshape(-1, 784))
        # ② 计算损失
        loss = loss_fn(y_hat, y)
        # ③ 反向传播
        optimizer.zero_grad() # 清空参数
        loss.backward() # 计算参数(参数保存在每个参数的grad属性中)
        optimizer.step() # 更新参数
    print(f"epoach: {epoch} loss: {loss.item()}")

epoach: 0 loss: 1.9524188041687012
epoach: 1 loss: 1.5578504800796509
epoach: 2 loss: 1.3717269897460938
epoach: 3 loss: 1.1581310033798218
epoach: 4 loss: 1.1286684274673462
epoach: 5 loss: 0.9818857312202454
epoach: 6 loss: 0.9169284701347351
epoach: 7 loss: 0.9693338871002197
epoach: 8 loss: 0.7670623660087585
epoach: 9 loss: 0.9085621237754822


### 模型推理预测

In [106]:
test_dl = DataLoader(test_data, batch_size=BATCH_SIZE)

total = 0
correct = 0

with torch.no_grad():  # 不计算梯度，节省内存
    for data, target in test_dl:  # data: 128*1*28*28（128张图片，1个通道，28*28），target: 128*1 （128个标签）
        output = model(data.reshape(-1, 784))
        # print(output.shape)  # 128*10
        max_out,max_idx = torch.max(output, dim=1) # 返回每行最大值和索引
        total += target.size(0) # 第0维的大小，即128 = shape(0)
        correct += (max_idx == target).sum().item() # 求和得到正确的个数

print(f"Accuracy: {correct/total}")

Accuracy: 0.7368


In [107]:
output

tensor([[ 1.9755e+00,  1.4738e+00, -9.4906e-01,  3.9483e+00,  1.2450e-01,
         -6.6696e-01,  1.0293e+00, -1.8182e+00, -2.5819e-01, -1.9695e+00],
        [ 8.4259e-01,  5.7607e-01,  9.6443e-01,  1.4058e+00,  1.6267e+00,
         -6.5534e-01,  1.6045e+00, -1.9628e+00, -2.7740e-01, -1.7215e+00],
        [-3.0917e+00, -1.8240e+00, -1.6697e+00, -9.2715e-01, -1.7685e+00,
          3.9932e+00, -1.2153e+00,  5.5386e+00,  1.8424e+00,  2.3527e+00],
        [-1.0389e+00, -1.9952e+00, -7.7800e-01, -9.5157e-01, -1.5831e+00,
          3.9125e+00, -5.2330e-02,  1.3266e+00,  1.1331e+00,  2.0917e+00],
        [-2.2465e+00, -2.9196e+00,  1.1203e-01, -1.4813e+00,  3.3248e-03,
          2.3692e+00,  2.5569e-01,  2.0034e+00,  3.9706e+00,  9.4605e-01],
        [ 7.4512e-01, -9.6655e-01,  3.4576e+00, -2.7982e-01,  3.4936e+00,
         -1.0004e+00,  3.0458e+00, -3.8110e+00,  4.0171e-01, -3.1317e+00],
        [ 6.8878e-01, -2.4629e+00, -1.3227e-01, -1.1828e+00, -1.8675e+00,
          3.7269e+00,  6.4393e-0